In [1]:
import torch
from pytorch_lightning import Trainer, LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import PreTrainedModel, BertTokenizer, BertConfig, BertForSequenceClassification
from transformers.optimization import AdamW
from Korpora import Korpora
import os, csv, re
from dataclasses import dataclass
from typing import List, Optional
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import ExponentialLR
from sklearn.model_selection import train_test_split
import pandas as pd 

In [2]:
!pwd

/tf/notebooks/NLP


In [3]:
trainDataDir = './train_data'
trainDataPath = os.path.join(trainDataDir, 'total_train.csv')
trainDataPath

'./train_data/total_train.csv'

In [4]:
trainCSVData = list(csv.reader(open(trainDataPath, "r", encoding="utf-8"), delimiter=",", quotechar='"'))[1:-1]
trainCSVData[:5]

[['지금 배달되나요?', '1', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['아 네 배달됩니다', '1', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['짬뽕류는 어떤 게 있나요? 잘 나가는 짬뽕 있나요?', '1', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['특해물 짬뽕도 있고 전복 새우 짬뽕도 있고 해물 종류도 새우 홍합 전복 없는 게 없습니다',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['전복 들어가는 거는 특해물 짬뽕 시켜야 돼요?', '1', '0', '0', '0', '0', '0', '0', '0', '0']]

In [5]:
trainCSVData[228363]

['아까 요 제품 이 저 는 좀 나은 거 같은 데 가격 은 똑같 아예 ?',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0']

위는 쉼표가 포함된 문장도 잘 불러와졌는지 한번 보려고 출력해본 것

In [6]:
train, test = train_test_split(trainCSVData, test_size=0.3)
'train', train[:2], len(train), 'test', test[:2], len(test)

('train',
 [['이 빵 은 오늘 만 든 거 예요', '0', '0', '0', '1', '0', '0', '0', '0', '0'],
  ['먹고 가 려면 한 줄 안 되 죠', '1', '0', '0', '0', '0', '0', '0', '0', '0']],
 253155,
 'test',
 [['메뉴 도 지금 정 할게요', '1', '0', '0', '0', '0', '0', '0', '0', '0'],
  ['아무 거나 사 용해도 되 나 요', '0', '0', '0', '0', '0', '0', '0', '1', '0']],
 108496)

In [7]:
@dataclass
class ClassificationDataFormat:
    text_a: str
    label: Optional[int] = None

In [8]:
ClassificationDataFormat(text_a=train[0][0], label=train[0][1:].index('1'))

ClassificationDataFormat(text_a='이 빵 은 오늘 만 든 거 예요', label=3)

In [9]:
[int(label) for label in train[0][1:]]

[0, 0, 0, 1, 0, 0, 0, 0, 0]

In [10]:
trainDataFormatList = []

In [11]:
for data in train:
    trainDataFormatList.append(ClassificationDataFormat(text_a=data[0], label=data[1:].index('1')))
trainDataFormatList[:5]

[ClassificationDataFormat(text_a='이 빵 은 오늘 만 든 거 예요', label=3),
 ClassificationDataFormat(text_a='먹고 가 려면 한 줄 안 되 죠', label=0),
 ClassificationDataFormat(text_a='카드 인가요', label=4),
 ClassificationDataFormat(text_a='신발 이 많이 쫄 린다', label=1),
 ClassificationDataFormat(text_a='최대 몇 층 건물 을 지을 수 있 어요', label=8)]

In [12]:
testDataFormatList = []

In [13]:
for data in test:
    testDataFormatList.append(ClassificationDataFormat(text_a=data[0], label=data[1:].index('1')))
testDataFormatList[:5]

[ClassificationDataFormat(text_a='메뉴 도 지금 정 할게요', label=0),
 ClassificationDataFormat(text_a='아무 거나 사 용해도 되 나 요', label=7),
 ClassificationDataFormat(text_a='주말과 평일 요금이 다른 가요?', label=6),
 ClassificationDataFormat(text_a='저희 아이 할만한 방이 어디 있을까요', label=4),
 ClassificationDataFormat(text_a='지난번 에 매직 은 언제 하셨 어요 ?', label=4)]

In [14]:
@dataclass
class ARGS:
    pretrained_model_name: str = 'beomi/kcbert-base'
    batch_size: int = 16
    learning_rate: float = 5e-5
    max_seq_length: int = 64
    epochs: int = 3
    tpu_cores: int = 0
    downstream_task_name: str = "chat-type-classification"
    cpu_workers: int = 7
    downstream_model_dir: str = './chatbot_model'
args = ARGS()

In [15]:
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name, #pre-trained model
    do_lower_case=False
)
tokenizer

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

PreTrainedTokenizer(name_or_path='beomi/kcbert-base', vocab_size=30000, model_max_len=300, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [16]:
batchEncoding = tokenizer(
    [data.text_a for data in trainDataFormatList],    
    max_length=args.max_seq_length, # 128
    padding="max_length", # 최대 길이 만큼 패딩
    truncation=True # 길이 오버시 자름 
)

In [17]:
batchEncoding.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [18]:
input_sample = {k: batchEncoding[k][0] for k in batchEncoding}

In [19]:
@dataclass
class ClassificationFeatures:
    input_ids: List[int]
    attention_mask: Optional[List[int]] = None
    token_type_ids: Optional[List[int]] = None
    label: Optional[List[int]] = None

In [20]:
feature_sample = ClassificationFeatures(**input_sample, label=trainDataFormatList[0].label)
feature_sample

ClassificationFeatures(input_ids=[2, 2451, 1692, 2420, 8564, 1296, 949, 248, 2289, 4040, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label=3)

In [21]:
train_feature = []

In [22]:
for idx in range(len(trainDataFormatList)):
    input_tokenized = {k: batchEncoding[k][idx] for k in batchEncoding}
    feature = ClassificationFeatures(**input_tokenized, label=trainDataFormatList[idx].label)
    train_feature.append(feature)
train_feature[:3]

[ClassificationFeatures(input_ids=[2, 2451, 1692, 2420, 8564, 1296, 949, 248, 2289, 4040, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label=3),
 ClassificationFeatures(input_ids=[2, 8558, 197, 1182, 4063, 3354, 2631, 2173, 900, 2613, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,

In [23]:
for idx, data in enumerate(trainDataFormatList[:3]):
    print(f"#{idx}: {data.text_a}")
    token = " /".join(tokenizer.convert_ids_to_tokens(train_feature[idx].input_ids))
    print(f"token: {token}\n========\n")

#0: 이 빵 은 오늘 만 든 거 예요
token: [CLS] /이 /빵 /은 /오늘 /만 /든 /거 /예 /##요 /[SEP] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD]

#1: 먹고 가 려면 한 줄 안 되 죠
token: [CLS] /먹고 /가 /려 /##면 /한 /줄 /안 /되 /죠 /[SEP] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD]

#2: 카드 인가요
token: [CLS] /카드 /인가요 /[SEP] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[

In [24]:
batchEncoding_for_test = tokenizer(
    [data.text_a for data in testDataFormatList],    
    max_length=args.max_seq_length, # 128
    padding="max_length", # 최대 길이 만큼 패딩
    truncation=True # 길이 오버시 자름 
)

In [25]:
test_feature = []

In [26]:
for idx in range(len(testDataFormatList)):
    input_tokenized = {k: batchEncoding_for_test[k][idx] for k in batchEncoding}
    feature = ClassificationFeatures(**input_tokenized, label=testDataFormatList[idx].label)
    test_feature.append(feature)
test_feature[:3]

[ClassificationFeatures(input_ids=[2, 22944, 867, 8001, 2539, 16669, 4040, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label=0),
 ClassificationFeatures(input_ids=[2, 8093, 29445, 1785, 2355, 8150, 900, 587, 2346, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [27]:
for idx, data in enumerate(testDataFormatList[:3]):
    print(f"#{idx}: {data.text_a}")
    token = " /".join(tokenizer.convert_ids_to_tokens(test_feature[idx].input_ids))
    print(f"token: {token}\n========\n")

#0: 메뉴 도 지금 정 할게요
token: [CLS] /메뉴 /도 /지금 /정 /할게 /##요 /[SEP] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD]

#1: 아무 거나 사 용해도 되 나 요
token: [CLS] /아무 /거나 /사 /용 /##해도 /되 /나 /요 /[SEP] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD] /[PAD]

#2: 주말과 평일 요금이 다른 가요?
token: [CLS] /주말 /##과 /평 /##일 /요금 /##이 /다른 /가 /##요 /? /[SEP] /[PAD] /[PAD]

In [28]:
def data_collator(features):
    batch = {}
    batch["input_ids"] = torch.tensor([feature.input_ids for feature in features], dtype=torch.long)
    batch["attention_mask"] = torch.tensor([feature.attention_mask for feature in features], dtype=torch.long)
    batch["token_type_ids"] = torch.tensor([feature.token_type_ids for feature in features], dtype=torch.long)
    batch["labels"] = torch.tensor([feature.label for feature in features], dtype=torch.long)
    return batch

In [29]:
data_collator(train_feature)

{'input_ids': tensor([[    2,  2451,  1692,  ...,     0,     0,     0],
         [    2,  8558,   197,  ...,     0,     0,     0],
         [    2, 11865, 21370,  ...,     0,     0,     0],
         ...,
         [    2, 15224,  4429,  ...,     0,     0,     0],
         [    2,   654, 10454,  ...,     0,     0,     0],
         [    2,  8225, 17837,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'labels': tensor([3, 0, 4,  ..., 3, 0, 0])}

In [30]:
trainDataLoader = DataLoader(
    train_feature,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_feature, replacement=False),
    collate_fn=data_collator,
    drop_last=False,
    num_workers=args.cpu_workers
)

In [31]:
testDataLoader = DataLoader(
    test_feature,
    batch_size=args.batch_size,
    sampler=RandomSampler(test_feature, replacement=False),
    collate_fn=data_collator,
    drop_last=False,
    num_workers=args.cpu_workers
)

In [32]:
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=9,
)

In [33]:
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [34]:
checkpointPath = args.downstream_model_dir
os.makedirs(checkpointPath, exist_ok=True)

In [35]:
checkpoint_callback = ModelCheckpoint(
        dirpath=checkpointPath,
        save_top_k=1,
        monitor="val_loss",
        mode="min",
        filename='{epoch}-{val_loss:.2f}',
    )

In [36]:
trainer = Trainer(
    max_epochs=args.epochs,
    fast_dev_run=False,
    num_sanity_val_steps=0,
    callbacks=[checkpoint_callback],
    default_root_dir=checkpointPath,
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [37]:
def accuracy(preds, labels, ignore_index=None):
    with torch.no_grad():
        assert preds.shape[0] == len(labels)
        correct = torch.sum(preds == labels)
        total = torch.sum(torch.ones_like(labels))
        if ignore_index is not None:
            # 모델이 맞춘 것 가운데 ignore index에 해당하는 것 제외
            correct -= torch.sum(torch.logical_and(preds == ignore_index, preds == labels))
            # accuracy의 분모 가운데 ignore index에 해당하는 것 제외
            total -= torch.sum(labels == ignore_index)
    return correct.to(dtype=torch.float) / total.to(dtype=torch.float)

In [38]:
class ClassificationTrainTask(LightningModule):
    def __init__(self, model: PreTrainedModel, args):
        super().__init__()
        self.model = model
        self.args = args
        
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.args.learning_rate)
        scheduler = ExponentialLR(optimizer, gamma=0.9)
        return {
            'optimizer': optimizer,
            'scheduler': scheduler
        }
    
    def training_step(self, inputs, batch_idx):
#         print('train input', inputs['labels'].size())
        outputs = self.model(**inputs)
#         print('train input', inputs['labels'].size(), outputs)
        predict = outputs.logits.argmax(dim=-1)
        labels = inputs['labels']
        acc = accuracy(predict, labels)
        self.log("loss", outputs.loss, prog_bar=False, logger=True, on_step=True, on_epoch=False)
        self.log("acc", acc, prog_bar=True, logger=True, on_step=True, on_epoch=False)
        return outputs.loss
        
    def validation_step(self, inputs, batch_idx):
#         print('inputs?', inputs)
        outputs = self.model(**inputs)
        predict = outputs.logits.argmax(dim=-1)
        labels = inputs["labels"]
        acc = accuracy(predict, labels)
        self.log("val_loss", outputs.loss, prog_bar=True, logger=True, on_step=False, on_epoch=True)
        self.log("val_acc", acc, prog_bar=True, logger=True, on_step=False, on_epoch=True)
        return outputs.loss

In [39]:
task = ClassificationTrainTask(model, args)

In [ ]:
trainer.fit(
    task,
    train_dataloader=trainDataLoader,
    val_dataloaders=testDataLoader
)


  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.702   Total estimated model params size (MB)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for val/test/predict dataloaders.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

Validating: 0it [00:00, ?it/s]